In [1]:
import os
os.chdir('..')

In [2]:
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import models
import scoring
from data import pronostia
import gc 
import pickle as pk
import pandas as pd

In [4]:
data1 = pd.read_csv('../data/pronostia/pronostia_train.csv')
data2 = pd.read_csv('../data/pronostia/pronostia_test.csv')


In [5]:
RUL = pd.concat((data1, data2))[['Bearing', 'Condition', 'RUL']]
del data1
del data2

In [6]:


MODELS = {
    'cnn_cnn_rna': ('e1bdbbece8a83a8965807683b4a1c454dc201a33', 
                    'net_cnn_l2_pronostia_embeddings_fold', 
                    'mscnn_pronostia', 110),
    'cnn_rnn_rna': ('2b8549cb2dca448b403924a090b3ee20ae96e536', 
                    'net_cnn_rnn_rna_pronostia_embeddings_fold', 
                    'mscnn_pronostia', 110),
    'rnn_rnn_rna': ('f6754f479fd9939af8fca7c4f8645a041bb96225',
                    'net_rnn_rnn_rna_pronostia_embeddings_fold',
                    'rnn_pronostia', 122),
    'rnn_cnn_rna': ('d6edaa0ccb4fd9fb38137e7c4f33730dab36fa27',
                    'net_rnn_cnn_rna_rnn_pronostia_embeddings_fold',
                    'rnn_pronostia', 122)

}



_dir = 'rnn_cnn_rna'

code, file, embeddings_dir, window = MODELS[_dir]

 

In [7]:
results = []

MAX_RULS = {(3, 1): 23749,
 (3, 2): 19549,
 (3, 3): 4339,
 (4, 1): 11728,
 (4, 2): 7509,
 (5, 1): 24629,
 (5, 2): 23109,
 (6, 1): 24479,
 (6, 2): 7009,
 (7, 1): 22589,
 (7, 2): 2299,
 (1, 1): 28029,
 (1, 2): 9109,
 (1, 3): 5149,
 (2, 1): 8709,
 (2, 2): 7969,
 (2, 3): 16369
}


for fold in range(5):

    rna = tf.keras.models.load_model(f'../results/nets/{_dir}/{code}/{file}_{fold}.h5',
                                    custom_objects={'LeakyReLU': tf.keras.layers.LeakyReLU,
                                                    'NASAScore': scoring.NASAScore,
                                                    'PHM21Score': scoring.PHM21Score,
                                                    'SelfAttention': models.SelfAttention})
    
    print(rna.inputs)
    channels = rna.inputs[0].shape[-1]
    if channels > 3:
        channels = 1
    num_embeddings = rna.inputs[0].shape[1] * channels
    
    test_bearings = [i for i in range(1, 8) if i not in pronostia.FOLD_TRAIN_BEARINGS[fold]]
    
    
    
    for bearing in test_bearings:
        embeddings_files = [f for f in os.listdir(f'../results/embeddings/{embeddings_dir}/fold_{fold}') 
                            if f.replace('.pk', '').split('_')[1] == str(bearing)]
        
        for embedding_file in embeddings_files:
            condition = int(embedding_file.replace('.pk', '').split('_')[0])

            embeddings = pk.load(open(f'../results/embeddings/{embeddings_dir}/fold_{fold}/{embedding_file}', 
                                      'rb'))
            
            
            rul = RUL[(RUL.Bearing == bearing) & (RUL.Condition == condition)].RUL.values
            t = [rul[i] for i in range(0, rul.shape[0], 100)][::-1]
            rul = [rul[i] / MAX_RULS[(bearing, condition)] for i in range(0, rul.shape[0], 100)]
            
            
            if len(rna.inputs[0].shape) == 3: #rnn
                X = np.array([embeddings[i:i+num_embeddings] for i in range(len(embeddings)-num_embeddings)])
                
            else: #  cnn
                X = np.array([embeddings[i:i+num_embeddings].T for i in range(len(embeddings)-num_embeddings)])
                #X = np.expand_dims(X, axis=-1)
                X = X.reshape((len(X),) + rna.inputs[0].shape[1:])
            
            T = [t[i+num_embeddings] for i in range(len(embeddings)-num_embeddings)]
            R = [rul[i+num_embeddings] for i in range(len(embeddings)-num_embeddings)]
            
            pred = rna.predict(X, batch_size=256, verbose=1)
            
            
            results.append({
                'fold': fold,
                'bearing': bearing,
                'condition': condition,
                'rul': R,
                'pred': pred,
                't': T
            })
            
            del embeddings
        
          
    del rna
    gc.collect()
    

pk.dump(results, open(f'../results/predictions/pronostia/{_dir}.pk', 'wb'))

[<tf.Tensor 'input_1:0' shape=(None, 100, 100, 3) dtype=float32>]
  2/230 [..............................] - ETA: 2:40WARNING:tensorflow:Method (on_predict_batch_end) is slow compared to the batch update (0.755798). Check your callbacks.


230/230 [==============================] - 21s 91ms/step
[<tf.Tensor 'input_1_1:0' shape=(None, 100, 100, 3) dtype=float32>]
  2/230 [..............................] - ETA: 8:32WARNING:tensorflow:Method (on_predict_batch_end) is slow compared to the batch update (1.804791). Check your callbacks.


57/57 [==============================] - 4s 75ms/step
[<tf.Tensor 'input_1_2:0' shape=(None, 100, 100, 3) dtype=float32>]
17/17 [==============================] - 1s 64ms/step
[<tf.Tensor 'input_1_3:0' shape=(None, 100, 100, 3) dtype=float32>]
17/17 [==============================] - 1s 65ms/step
[<tf.Tensor 'input_1_4:0' shape=(None, 100, 100, 3) dtype=float32>]
  2/230 [..............................] - ETA: 6:51WARNING:tensorflow:Method (on_predict_batch_end) is slow compared to the batch update (1.987071). Check your callbacks.


57/57 [==============================] - 5s 84ms/step
